In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_sprint_speed(player_id):
    # URL for the player's Baseball Savant page (Running Stats)
    url = f"https://baseballsavant.mlb.com/savant-player/{player_id}?stats=statcast-r-running-mlb"
    
    # Send GET request to fetch the page content
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch page: {response.status_code}")
    
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the table containing sprint speed data
    table = soup.find('table', {'class': 'player-stats-table'})
    if table is None:
        raise Exception("Sprint speed table not found on the page.")
    
    # Extract the table rows
    rows = table.find('tbody').find_all('tr')
    data = []
    for row in rows:
        cells = row.find_all('td')
        year = cells[0].text.strip()
        sprint_speed = cells[1].text.strip()
        percentile = cells[2].text.strip()
        data.append({'Year': year, 'Sprint Speed (ft/sec)': sprint_speed, 'Percentile': percentile})
    
    # Convert to a Pandas DataFrame
    return pd.DataFrame(data)

# Example Usage
player_id = "545361"  # Mike Trout's Baseball Savant ID
try:
    sprint_speed_data = get_sprint_speed(player_id)
    print(sprint_speed_data)
except Exception as e:
    print(f"Error: {e}")


Error: Sprint speed table not found on the page.


In [ ]:
def get_pitch_by_pitch_data(game_id):
    if game_id is None:
        print("Invalid game ID.")
        return []

    url = f"https://statsapi.mlb.com/api/v1/game/{game_id}/playByPlay"
    response = requests.get(url)
    data = response.json()

    if 'allPlays' not in data:
        print("No play-by-play data available.")
        return []

    pitches_data = []
    for play in data['allPlays']:
        if 'playEvents' in play:
            for pitch in play['playEvents']:
                if pitch.get('details', {}).get('type', {}).get('code') == 'pitch':
                    result = {
                        "batter": play['matchup']['batter']['fullName'],
                        "pitcher": play['matchup']['pitcher']['fullName'],
                        "pitch_type": pitch['details'].get('pitchType', {}).get('description', 'Unknown'),
                        "pitch_speed": pitch.get('pitchData', {}).get('startSpeed', 'N/A'),
                        "description": pitch['details']['description'],
                        "event": play['result']['event'],
                        "event_type": play['result']['eventType'],
                        "date": data['gameData']['datetime']['dateTime'],
                    }
                    pitches_data.append(result)

    return pitches_data

# Example: Get pitch-by-pitch data for the retrieved game ID
pitch_data = get_pitch_by_pitch_data(game_id)

# Print the first few pitch events
for pitch in pitch_data[:5]:
    print(pitch)


In [5]:
url = f"https://statsapi.mlb.com/api/v1/game/745337/playByPlay"
response = requests.get(url)
data = response.json()

In [6]:
# runners

start_base = data['allPlays'][18]['runners']['movement']['start']

start_base = data['allPlays'][18]['runners']['movement']['end']

event = data['allPlays'][18]['runners']['movement']['details']['event']

runner_id = data['allPlays'][18]['runners']['movement']['details']['runner']['id']

runner_name = data['allPlays'][18]['runners']['movement']['details']['runner']['fullName']


TypeError: list indices must be integers or slices, not str

In [12]:
data['allPlays'][18]['matchup']['splits']['menOnBase']

'Empty'

In [124]:
event

{'movement': {'originBase': None,
  'start': None,
  'end': None,
  'outBase': '1B',
  'isOut': True,
  'outNumber': 3},
 'details': {'event': 'Flyout',
  'eventType': 'field_out',
  'movementReason': None,
  'runner': {'id': 150421,
   'fullName': 'Ramon Hernandez',
   'link': '/api/v1/people/150421'},
  'responsiblePitcher': None,
  'isScoringEvent': False,
  'rbi': False,
  'earned': False,
  'teamUnearned': False,
  'playIndex': 4},
 'credits': [{'player': {'id': 115915, 'link': '/api/v1/people/115915'},
   'position': {'code': '7',
    'name': 'Outfielder',
    'type': 'Outfielder',
    'abbreviation': 'LF'},
   'credit': 'f_putout'}]}

In [44]:
import requests

# Define parameters
date = "2024-05-28"
team_ids = [137, 144]  # Example: Giants (137) and Phillies (144)

# Schedule endpoint
url = "https://statsapi.mlb.com/api/v1/schedule"
params = {
    "sportId": 1,  # Major League Baseball
    "date": date
}

# Make the API request
response = requests.get(url, params=params)
data = response.json()

# Extract game ID for the specified teams
game_id = None
if "dates" in data and len(data["dates"]) > 0:
    for game in data["dates"][0]["games"]:
        home_team = game["teams"]["home"]["team"]["id"]
        away_team = game["teams"]["away"]["team"]["id"]
        if home_team in team_ids and away_team in team_ids:
            game_id = game["gamePk"]
            print(f"Game ID: {game_id}")
            break

if not game_id:
    print(f"No game found for teams {team_ids} on {date}.")


No game found for teams [137, 144] on 2024-05-28.


In [130]:
data

{'copyright': 'Copyright 2024 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'allPlays': [{'result': {'type': 'atBat',
    'event': 'Pop Out',
    'eventType': 'field_out',
    'description': 'Kyle Schwarber pops out to third baseman Matt Chapman.',
    'rbi': 0,
    'awayScore': 0,
    'homeScore': 0,
    'isOut': True},
   'about': {'atBatIndex': 0,
    'halfInning': 'top',
    'isTopInning': True,
    'inning': 1,
    'startTime': '2024-05-29T01:46:05.684Z',
    'endTime': '2024-05-29T01:46:48.064Z',
    'isComplete': True,
    'isScoringPlay': False,
    'hasReview': False,
    'hasOut': True,
    'captivatingIndex': 0},
   'count': {'balls': 1, 'strikes': 1, 'outs': 1},
   'matchup': {'batter': {'id': 656941,
     'fullName': 'Kyle Schwarber',
     'link': '/api/v1/people/656941'},
    'batSide': {'code': 'L', 'description': 'Left'},
    'pitcher': {'id': 669062,
     'fullNam

In [48]:
runners_list = []


for atbat in data['allPlays']:
    
    for event in atbat['runners']:
    
        # runners
        
        runners_list.append({
            'Start_Base': event['movement']['start'],
            'End_Base': event['movement']['end'],
            'Event': event['details']['event'],
            'Respon_Pitcher': event['details']['responsiblePitcher'],
            'runner_id' : event['details']['runner']['id'],
            'runner_name' : event['details']['runner']['fullName']
        })


In [49]:
runners_df = pd.DataFrame(runners_list)
runners_df.head()

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name
0,None,None,Pop Out,None,656941,Kyle Schwarber
1,None,None,Strikeout,None,592663,J.T. Realmuto
2,None,1B,Walk,None,547180,Bryce Harper
3,None,None,Strikeout,None,592206,Nick Castellanos
4,None,None,Strikeout,None,682641,Luis Matos


In [50]:
runners_df.loc[runners_df['Event'] == 'Stolen Base 2B']

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name


In [51]:
runners_df['Event'].unique()

array(['Pop Out', 'Strikeout', 'Walk', 'Forceout', 'Groundout',
       'Grounded Into DP', 'Flyout', 'Single', 'Strikeout Double Play',
       'Caught Stealing 2B', 'Lineout', 'Double', 'Fielders Choice Out',
       'Wild Pitch', 'Intent Walk', 'Sac Fly'], dtype=object)

In [52]:
runners_df.loc[runners_df['Event'] == 'Caught Stealing 2B']

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name
38,1B,None,Caught Stealing 2B,None,679032,Johan Rojas


In [14]:
runners_list = []


for atbat in data['allPlays']:
    
    for event in atbat['runners']:
    
        # runners
        
        if event['details']['event'].split(' ')[0] == 'Caught':
            credit = str(event['credits'][0]['player']['id'])
        else:
            credit = None
        
        runners_list.append({
            'Start_Base': event['movement']['start'],
            'End_Base': event['movement']['end'],
            'Event': event['details']['event'],
            'Respon_Pitcher': event['details']['responsiblePitcher'],
            'runner_id' : event['details']['runner']['id'],
            'runner_name' : event['details']['runner']['fullName'],
            'credit': credit,
            'men_on': atbat['matchup']['splits']['menOnBase']
        })


In [15]:
runners_df = pd.DataFrame(runners_list)
runners_df.head()

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name,credit,men_on
0,None,None,Pop Out,None,656941,Kyle Schwarber,None,Empty
1,None,None,Strikeout,None,592663,J.T. Realmuto,None,Empty
2,None,1B,Walk,None,547180,Bryce Harper,None,Men_On
3,None,None,Strikeout,None,592206,Nick Castellanos,None,Empty
4,None,None,Strikeout,None,682641,Luis Matos,None,Empty


In [20]:
data.keys()

dict_keys(['copyright', 'allPlays', 'currentPlay', 'scoringPlays', 'playsByInning'])

In [23]:
data['playsByInning']

[{'startIndex': 0,
  'endIndex': 7,
  'top': [0, 1, 2, 3],
  'bottom': [4, 5, 6, 7],
  'hits': {'away': [{'team': {'springLeague': {'id': 115,
       'name': 'Grapefruit League',
       'link': '/api/v1/league/115',
       'abbreviation': 'GL'},
      'allStarStatus': 'N',
      'id': 143,
      'name': 'Philadelphia Phillies',
      'link': '/api/v1/teams/143'},
     'inning': 1,
     'pitcher': {'id': 669062,
      'fullName': 'Erik Miller',
      'link': '/api/v1/people/669062'},
     'batter': {'id': 656941,
      'fullName': 'Kyle Schwarber',
      'link': '/api/v1/people/656941'},
     'coordinates': {'x': 112.77, 'y': 166.56},
     'type': 'O',
     'description': 'Pop Out'}],
   'home': [{'team': {'springLeague': {'id': 114,
       'name': 'Cactus League',
       'link': '/api/v1/league/114',
       'abbreviation': 'CL'},
      'allStarStatus': 'N',
      'id': 137,
      'name': 'San Francisco Giants',
      'link': '/api/v1/teams/137'},
     'inning': 1,
     'pitcher': {'id'

In [129]:
runners_df.loc[runners_df['Event'] == 'Caught Stealing 2B']

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name,credit
38,1B,None,Caught Stealing 2B,None,679032,Johan Rojas,672275


In [122]:
runners_df['Action'] = runners_df['Event'].apply(lambda x: x.split(' ')[0])

runners_df['Base'] = runners_df['Event'].apply(lambda x: x.split(' ')[-1])

In [123]:
runners_df = runners_df.loc[(runners_df['Action'] == 'Caught') | (runners_df['Action'] == 'Stolen')]

In [124]:
runners_df

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name,credit,Action,Base
38,1B,None,Caught Stealing 2B,None,679032,Johan Rojas,672275,Caught,2B


In [3]:
 import requests
from datetime import datetime, timedelta

def get_all_game_ids(start_year, end_year):
    base_url = "https://statsapi.mlb.com/api/v1/schedule"
    
    game_list = []

    while start_year != end_year:

        params = {
            "sportId": 1,  # 1 is for MLB
            "startDate": f"{start_year}-04-01",
            "endDate": f"{start_year}-09-30",
        }

        response = requests.get(base_url, params=params)
        data = response.json()

        games = data.get("dates", [])

        for game_day in games:
            date = game_day["date"]
            for game in game_day['games']:

                try:
                    game_list.append({
                    "gameId": game["gamePk"],
                    "gameDate": date,
                    "homeTeam": game["teams"]["home"]["team"]["name"],
                    "awayTeam": game["teams"]["away"]["team"]["name"],
                    "awayTeamID": game['teams']['away']['team']['id'],
                    "homeTeamID": game['teams']['home']['team']['id'],})
                except:

                    game_list.append({
                    "gameId": game["gamePk"],
                    "gameDate": date,
                    "homeTeam": np.nan,
                    "awayTeam": np.nan,
                    "awayTeamID": game['teams']['away']['team']['id'],
                    "homeTeamID": game['teams']['home']['team']['id'],
                })
        start_year += 1  


    return game_list


# Example: Get all game IDs and dates for the 2023 MLB regular season
start_year = 2000
end_year = 2024

game_list = get_all_game_ids(start_year, end_year)

# Print the results
for game in game_list[:10]:  # Print the first 10 games as a sample
    print(game)


{'gameId': 3930, 'gameDate': '2000-04-03', 'homeTeam': 'Cincinnati Reds', 'awayTeam': 'Milwaukee Brewers', 'awayTeamID': 158, 'homeTeamID': 113}
{'gameId': 3931, 'gameDate': '2000-04-03', 'homeTeam': 'New York Mets', 'awayTeam': 'San Diego Padres', 'awayTeamID': 135, 'homeTeamID': 121}
{'gameId': 3409, 'gameDate': '2000-04-03', 'homeTeam': 'Baltimore Orioles', 'awayTeam': 'Cleveland Indians', 'awayTeamID': 114, 'homeTeamID': 110}
{'gameId': 3410, 'gameDate': '2000-04-03', 'homeTeam': 'Texas Rangers', 'awayTeam': 'Chicago White Sox', 'awayTeamID': 145, 'homeTeamID': 140}
{'gameId': 3411, 'gameDate': '2000-04-03', 'homeTeam': 'Toronto Blue Jays', 'awayTeam': 'Kansas City Royals', 'awayTeamID': 118, 'homeTeamID': 141}
{'gameId': 3932, 'gameDate': '2000-04-03', 'homeTeam': 'Atlanta Braves', 'awayTeam': 'Colorado Rockies', 'awayTeamID': 115, 'homeTeamID': 144}
{'gameId': 3933, 'gameDate': '2000-04-03', 'homeTeam': 'St. Louis Cardinals', 'awayTeam': 'Chicago Cubs', 'awayTeamID': 112, 'homeTe

In [4]:
data_df = pd.DataFrame(game_list)

gameids = data_df['gameId'].unique().tolist()

In [5]:
data_df

,gameId,gameDate,homeTeam,awayTeam,awayTeamID,homeTeamID
0,3930,2000-04-03,Cincinnati Reds,Milwaukee Brewers,158,113
1,3931,2000-04-03,New York Mets,San Diego Padres,135,121
2,3409,2000-04-03,Baltimore Orioles,Cleveland Indians,114,110
3,3410,2000-04-03,Texas Rangers,Chicago White Sox,145,140
4,3411,2000-04-03,Toronto Blue Jays,Kansas City Royals,118,141
...,...,...,...,...,...,...
57557,716369,2023-09-30,Atlanta Braves,Washington Nationals,120,144
57558,716370,2023-09-30,Arizona Diamondbacks,Houston Astros,117,109
57559,716368,2023-09-30,Colorado Rockies,Minnesota Twins,142,115
57560,716381,2023-09-30,San Francisco Giants,Los Angeles Dodgers,119,137


In [ ]:
df_final = pd.DataFrame()

for row, i in data_df.iterrows():
    
    game = i['gameId']

    boxscore_url = f"https://statsapi.mlb.com/api/v1/game/{game}/boxscore"
    boxscore_response = requests.get(boxscore_url).json()

    # Step 3: Extract the lineup
    home_team = boxscore_response['teams']['home']
    away_team = boxscore_response['teams']['away']

    home_lineup = []

    home_catcher = ''

    away_lineup = []

    away_catcher = ''

    for player_id, player_data in home_team['players'].items():
        if 'battingOrder' in player_data:

            home_lineup.append(player_data['person']['fullName'])

            if player_data['position']['abbreviation'] == 'C':

                home_catcher = player_data['person']['fullName']

    for player_id, player_data in away_team['players'].items():
        if 'battingOrder' in player_data:
            away_lineup.append(player_data['person']['fullName'])

            if player_data['position']['abbreviation'] == 'C':

                away_catcher = player_data['person']['fullName']


    url = f"https://statsapi.mlb.com/api/v1/game/{game}/playByPlay"
    response = requests.get(url)
    data = response.json()
    
    if len(data['allPlays']) == 0:
        continue
    
    runners_list = []


    for atbat in data['allPlays']:

        for event in atbat['runners']:

            # runners
            
            if event['details']['runner']['fullName'] in home_lineup:
                credit = away_catcher
            else:
                credit = home_catcher


            runners_list.append({
                'Start_Base': event['movement']['start'],
                'End_Base': event['movement']['end'],
                'Event': event['details']['event'],
                'Respon_Pitcher': event['details']['responsiblePitcher'],
                'runner_id' : event['details']['runner']['id'],
                'runner_name' : event['details']['runner']['fullName'],
                'credit': credit,
                'homeScore': atbat['result']['homeScore'],
                'awayScore': atbat['result']['awayScore'],
                'inning': atbat['about']['inning'],
                'isTopInning': atbat['about']['isTopInning'],
                
            })
    
    runners_df = pd.DataFrame(runners_list)
    
    runners_df['Date'] = i['gameDate']
    
    runners_df['Action'] = runners_df['Event'].apply(lambda x: x.split(' ')[0])

    runners_df['Base'] = runners_df['Event'].apply(lambda x: x.split(' ')[-1])
    
    runners_df = runners_df.loc[(runners_df['Action'] == 'Caught') | (runners_df['Action'] == 'Stolen')]
    
    df_final = pd.concat([runners_df, df_final])


In [113]:
away_lineup

['Nick Maton',
 'Zach McKinstry',
 'Austin Meadows',
 'Javier Báez',
 'Riley Greene',
 'Kerry Carpenter',
 'Spencer Torkelson',
 'Ryan Kreidler',
 'Eric Haase',
 'Matt Vierling']

In [117]:
df_final

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name,credit,Date,Action,Base
49,1B,2B,Stolen Base 2B,None,114789,Brian Giles,Ramon Castro,2004-05-07,Stolen,2B
58,1B,2B,Stolen Base 2B,None,150239,Terrence Long,Ramon Castro,2004-05-07,Stolen,2B
56,3B,None,Caught Stealing Home,None,123744,Omar Vizquel,Javy Lopez,2004-05-07,Caught,Home
60,1B,2B,Stolen Base 2B,None,406878,Brian Roberts,Tim Laker,2004-05-07,Stolen,2B
80,1B,2B,Stolen Base 2B,None,117566,Matt Lawton,Javy Lopez,2004-05-07,Stolen,2B
...,...,...,...,...,...,...,...,...,...,...
63,1B,2B,Stolen Base 2B,None,136866,Carlos Febles,Darrin Fletcher,2000-04-03,Stolen,2B
81,1B,2B,Stolen Base 2B,None,124748,Gregg Zaun,Darrin Fletcher,2000-04-03,Stolen,2B
39,1B,None,Caught Stealing 2B,None,112345,Will Clark,Sandy Alomar Jr.,2000-04-03,Caught,2B
50,1B,None,Caught Stealing 2B,None,115815,Carlos Hernandez,Mike Piazza,2000-04-03,Caught,2B


In [105]:
df_final.sort_values(by='Date')

,Start_Base,End_Base,Event,Respon_Pitcher,runner_id,runner_name,credit,Date,Action,Base
26,1B,2B,Stolen Base 2B,None,115174,Marquis Grissom,Yainer Diaz,2000-04-03,Stolen,2B
1,1B,None,Caught Stealing 2B,None,117197,Chuck Knoblauch,Yainer Diaz,2000-04-03,Caught,2B
17,1B,None,Caught Stealing 2B,None,136267,Troy Glaus,Yainer Diaz,2000-04-03,Caught,2B
30,1B,None,Caught Stealing 2B,None,120028,Paul O'Neill,Yainer Diaz,2000-04-03,Caught,2B
34,1B,2B,Stolen Base 2B,None,150217,Cristian Guzman,Yainer Diaz,2000-04-03,Stolen,2B
...,...,...,...,...,...,...,...,...,...,...
9,1B,2B,Stolen Base 2B,None,114935,Luis Gonzalez,Yainer Diaz,2003-04-07,Stolen,2B
8,1B,None,Caught Stealing 2B,None,150396,Milton Bradley,Yainer Diaz,2003-04-08,Caught,2B
94,1B,None,Caught Stealing 2B,None,119247,Raul Mondesi,Yainer Diaz,2003-04-08,Caught,2B
7,1B,2B,Stolen Base 2B,None,276376,Nick Johnson,Yainer Diaz,2003-04-08,Stolen,2B


In [109]:
df_final['credit'].unique()

array(['Yainer Diaz'], dtype=object)